## Исследование производительности работы с хранилищем MongoDB

In [8]:
import uuid
import time

from typing import Any
from functools import wraps
from random import randint, choice
from motor.motor_asyncio import AsyncIOMotorClient

In [9]:
client = AsyncIOMotorClient('mongodb://localhost:27019,localhost:27020')
database = client["moviesDb"]
collection = database["movies"]

In [10]:
def benchmark(operation: str):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            start_time = time.perf_counter()
            func_result = await func(*args, **kwargs)
            end_time = time.perf_counter()
            avg_time = end_time - start_time
            print(f"Время выполнения функции {operation} заняло {avg_time} cек.")
            return func_result
        return wrapper
    return decorator

In [11]:
def generate_like():
    return {
        'event_name': 'like',
        'user_id': str(uuid.uuid4()),
        'movie_id': str(uuid.uuid4()),
        'score': randint(0, 10)
    }


def generate_bookmark():
    return {
        'event_name': 'bookmark',
        'user_id': str(uuid.uuid4()),
        'movie_id': str(uuid.uuid4())
    }


async def generate_data_in_mongo(
    generator: Any,
    docs_count: int,
    batch_size: int = 100
):
    docs = []
    for _ in range(docs_count):
        docs.append(generator())
        if len(docs) >= batch_size:
            await collection.insert_many(docs)
            docs = []
    if docs:
        await collection.insert_many(docs)


async def get_random_event_docs(
    event_name: str,
    docs_count: int = 10
) -> list[dict]:
    return await collection.aggregate(
		[
			{'$match': {'event_name': event_name}},
            {'$sample': {'size': docs_count}},
        ]
    ).to_list(docs_count)

In [12]:
@benchmark(operation='Получение количества лайков для конкретного фильма')
async def get_count_of_likes_for_movie(
    movie_id: uuid.UUID
) -> int:
    return await collection.count_documents(
        {'event_name': 'like', 'movie_id': str(movie_id)}
    )


@benchmark(operation='Получение списка закладок для рандомного пользователя')
async def get_user_bookmarks(
    user_id: uuid.UUID,
) -> list:
    cursor = collection.find(
		{'event_name': 'bookmark', 'user_id': str(user_id)}
    )
    return [doc async for doc in cursor]


@benchmark(operation='Добавление лайка пользователя для фильма')
async def add_like_to_movie(
    user_id: uuid.UUID,
    movie_id: uuid.UUID
) -> None:
    await collection.insert_one(
		{
            'event_name': 'like',
            'user_id': str(user_id),
            'movie_id': str(movie_id)
        }
    )


@benchmark(operation='Удаление лайка пользователя для фильма')
async def delete_like_from_movie(
    user_id: uuid.UUID,
    movie_id: uuid.UUID
) -> None:
    await collection.delete_one(
		{
            'event_name': 'like',
            'user_id': str(user_id),
            'movie_id': str(movie_id)
        }
    )

In [13]:
await generate_data_in_mongo(generate_like, 100_000, 500)
await generate_data_in_mongo(generate_bookmark, 100_000, 500)

random_like_docs = await get_random_event_docs('like')
random_bookmark_docs = await get_random_event_docs('bookmark')

await get_count_of_likes_for_movie(choice(random_like_docs)['movie_id'])
await get_user_bookmarks(choice(random_bookmark_docs)['user_id'])

random_user_id, random_movie_id = (uuid.uuid4(), uuid.uuid4())
await add_like_to_movie(random_user_id, random_movie_id)
await delete_like_from_movie(
    choice(random_like_docs)['user_id'],
    choice(random_bookmark_docs)['movie_id']
)

Время выполнения функции Получение количества лайков для конкретного фильма заняло 0.00999901500017586 cек.
Время выполнения функции Получение списка закладок для рандомного пользователя заняло 0.2051694870006031 cек.
Время выполнения функции Добавление лайка пользователя для фильма заняло 0.023002511999948183 cек.
Время выполнения функции Удаление лайка пользователя для фильма заняло 0.008686141999532992 cек.


### Вывод
Скорость чтения данных из MongoDB укладывается в требования (200мс)